In [31]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import math

### Data

In [32]:
sortvar = "hue"

In [33]:
DIR = "/Users/damoncrockett/Desktop/"

In [34]:
df = pd.read_csv(DIR+"phew_hsv.csv")

### Point list in polar

In [35]:
n = len(df)

In [36]:
r = int(math.ceil(np.sqrt(n/np.pi)))

In [37]:
a = []
for radius in range(1,r+1):
    area = radius * radius * np.pi
    a.append(area)

In [38]:
sizes = [int(round(a[i]-a[i-1])) for i in range(1,len(a))]

In [39]:
sizes.insert(0,int(round(a[0])))

In [40]:
radii = range(1,r+1)

In [41]:
all_rho = []
all_phi = []
for i in range(len(radii)):
    radius = radii[i]
    size = sizes[i]
    radstep = 2*np.pi / size # circumference is 2pi radians
    
    phi_offset = np.random.choice(np.arange(-np.pi,np.pi,np.pi/1024)) # we'll use this to jitter our starting pt
    phis_unjittered = np.arange(0,2*np.pi,radstep) # basic polar angle range
    
    phis = [phi+phi_offset for phi in phis_unjittered] # and of course offset needs to be applied to all    
    phis_corrected = [] # here we will keep the [0,2pi] range despite the jitter
    
    for phi in phis:
        if phi > 2*np.pi:
            xtra = phi - 2*np.pi
            phis_corrected.append(xtra)
        elif phi < 0:
            boosted = phi + 2*np.pi
            phis_corrected.append(boosted)
        else:
            phis_corrected.append(phi)
    
    all_rho.append(np.repeat(radius,len(phis_corrected)))
    all_phi.append([item for item in phis_corrected])
all_rho = [item for sublist in all_rho for item in sublist]
all_phi = [item for sublist in all_phi for item in sublist]

In [42]:
len(all_phi)

869223

In [43]:
len(all_rho)

869223

In [44]:
df.sort_values(by=sortvar,inplace=True)

In [45]:
tmp = pd.DataFrame({"rho":all_rho,"phi":all_phi})

In [46]:
tmp = tmp.iloc[:len(df),:] # we cut off extra positions we don't need bc our data is smaller than point set

In [47]:
len(tmp)==len(df)

True

### Create phi bins, attach to hue-sorted df, sort by phibin and sat, attach ordered rhos

In [48]:
phibins = int(round( len(df) / r )) # total pts divided by radius

In [49]:
tmp['phibin'] = pd.cut(tmp.phi,phibins,labels=False)

In [50]:
tmp.sort_values(by=['phibin','rho'],inplace=True)

In [51]:
tmp.head()

,phi,rho,phibin
152,0.000299,7,0
3495,0.002834,34,0
3822,0.000551,35,0
4397,0.000260,38,0
10360,0.000969,58,0


In [52]:
df['phibin'] = list(tmp.phibin)
df['phi'] = list(tmp.phi)

In [53]:
df.sort_values(by=['phibin','sat'],ascending=[True,False],inplace=True) # we want most sat in center

In [54]:
df['rho'] = list(tmp.rho)

### Plot

In [63]:
from PIL import Image

In [64]:
thumb_side = 16

In [65]:
px_w = (r * 2 * thumb_side) + thumb_side
px_h = px_w
print px_w, px_h

16848 16848


In [66]:
canvas = Image.new('RGB',(px_w,px_h),'hsl(180,0%,100%)') # white canvas

#### I find that the conversion to xy coords, the translation to image positions, and accounting for img size is best saved for right before plotting, to keep things straight

In [67]:
def pol2cart(rho, phi):
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return(x, y)

In [68]:
for i in df.index:
    binnum = df.phibin.loc[i]
    if np.random.binomial(1,0.00001)==1:
        print binnum
    
    im = Image.open(df.local_path.loc[i])
    im = im.convert('RGBA')
    im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
    
    rho = df.rho.loc[i]
    phi = df.phi.loc[i]    
    im = im.rotate(phi,expand=1)

    xgrid = pol2cart(rho,phi)[0]
    ygrid = pol2cart(rho,phi)[1]
    
    xcoord = int(round( (xgrid+r) * thumb_side) ) # r here is doing simple spatial translation
    ycoord = int(round( (r-ygrid) * thumb_side) ) #bc high in the image is low y
    canvas.paste(im,(xcoord,ycoord),im) # tmp treated as a mask for itself

385
414
449
479
480
576
616
766
1130
1160
1398
1420
1475
1507
1600
1644
1647


In [69]:
canvas.save(DIR+"firstlook-flat.png")